In [2]:
import pandas as pd
import json
import os
import logging
logging.basicConfig(level=logging.INFO, 
                    # filename=logfile, 
                    format='%(asctime)s %(levelname)s %(message)s',datefmt='%Y-%m-%d %H:%M:%S')

In [3]:
def export_alpha_sig(vector_artifact, metadata_md):
    import qiime2.plugins.diversity.actions as diversity_actions
    viz, = diversity_actions.alpha_group_significance(
        alpha_diversity=vector_artifact,
        metadata=metadata_md,
    )
    return viz
    

In [4]:
def get_alpha_vectors_sig(alpha_index_csv, metadata_csv, outdir):
    from qiime2 import Artifact
    from qiime2 import Metadata
    metadata_md = Metadata.load(metadata_csv)
    df_vectors = pd.read_csv(alpha_index_csv, index_col=0)
    metrics = ['faith_pd', 'observed_features', 'shannon_entropy' ,'pielou_evenness']
    for m in metrics:
        if m in df_vectors.columns:
            tmp_vector = Artifact.import_data('SampleData[AlphaDiversity]', df_vectors[m])
            tmp_viz = export_alpha_sig(tmp_vector, metadata_md)
            tmp_viz.export_data(os.path.join(outdir, m))
        else:
            logging.info(f'{m} not in {alpha_index_csv} columns')
        

In [5]:
json_file = 'test_out/merge_out/Amplicon_merge_with_diversity.json'

with open(json_file, 'rt') as H:
    merge_info = json.load(H)
alpha_index_csv = merge_info['alpha_index_csv']

outdir = 'test_out/div_sig/'
metadata_csv = 'test_out/div_sig/metadata.csv'
get_alpha_vectors_sig(alpha_index_csv, metadata_csv, outdir)

In [22]:
def export_ancom_sig(table_artifact, metadata_categroy):
    import qiime2.plugins.composition.actions as composition_actions
    comp_table, = composition_actions.add_pseudocount(
        table=table_artifact,
    )
    ancom_viz, = composition_actions.ancom(
        table=comp_table,
        metadata=metadata_categroy,
    )
    return ancom_viz

def get_ancom_sig(table, metadata, outdir):
    # from qiime2 import Artifact
    # from qiime2 import Metadata
    # metadata_md = Metadata.load(metadata_csv)
    # table = Artifact.load(table_qza)
    
    for group in metadata.columns:
        logging.info(f'get_ancom_sig: {group}')
        try:
            ancom_viz = export_ancom_sig(table, metadata.get_column(group))
            ancom_viz.export_data(os.path.join(outdir, f'ancom_{group}'))
        except Exception as e:
            logging.error(e)
            
def get_ancom_sig_taxa(table_qza, metadata_csv, tax_qza, outdir):
    from qiime2 import Artifact
    from qiime2 import Metadata
    import qiime2.plugins.taxa.actions as taxa_actions
    metadata_md = Metadata.load(metadata_csv)
    table = Artifact.load(table_qza)
    taxonomy = Artifact.load(tax_qza)
    # for i in range(7):
    tax_table, = taxa_actions.collapse(
        table=table,
        taxonomy=taxonomy,
        level=6,
    )
    get_ancom_sig(tax_table, metadata_md, outdir)
    


In [23]:
json_file = 'test_out/merge_out/Amplicon_merge_with_diversity.json'

with open(json_file, 'rt') as H:
    merge_info = json.load(H)
table_qza = merge_info['merged_tab_qza']
tax_qza = merge_info['merged_tax_qza']
outdir = 'test_out/div_sig/'
metadata_csv = 'test_out/div_sig/metadata.csv'
get_ancom_sig_taxa(table_qza, metadata_csv, tax_qza, outdir)

2022-04-12 17:44:28 INFO get_ancom_sig: sample
2022-04-12 17:44:28 ERROR All values in `grouping` are unique. This method cannot operate on a grouping vector with only unique values (e.g., there are no 'within' variance because each group of samples contains only a single sample).
2022-04-12 17:44:28 INFO get_ancom_sig: group
/home/yangk/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2022-04-12 17:44:39 INFO get_ancom_sig: 11
/home/yangk/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
